**1488. Avoid Flood in The City**

**Medium**

**Companies** : Google

Your country has an infinite number of lakes. Initially, all the lakes are empty, but when it rains over the nth lake, the nth lake becomes full of water. If it rains over a lake that is full of water, there will be a flood. Your goal is to avoid floods in any lake.

Given an integer array rains where:

- rains[i] > 0 means there will be rains over the rains[i] lake.
- rains[i] == 0 means there are no rains this day and you can choose one lake this day and dry it.
Return an array ans where:

- ans.length == rains.length
- ans[i] == -1 if rains[i] > 0.
- ans[i] is the lake you choose to dry in the ith day if rains[i] == 0.
  
If there are multiple valid answers return any of them. If it is impossible to avoid flood return an empty array.

Notice that if you chose to dry a full lake, it becomes empty, but if you chose to dry an empty lake, nothing changes.

 

**Example 1:**
```python
Input: rains = [1,2,3,4]
Output: [-1,-1,-1,-1]
```
**Explanation:** After the first day full lakes are [1]
After the second day full lakes are [1,2]
After the third day full lakes are [1,2,3]
After the fourth day full lakes are [1,2,3,4]
There's no day to dry any lake and there is no flood in any lake.

**Example 2:**
```python
Input: rains = [1,2,0,0,2,1]
Output: [-1,-1,2,1,-1,-1]
```
**Explanation:** After the first day full lakes are [1]
After the second day full lakes are [1,2]
After the third day, we dry lake 2. Full lakes are [1]
After the fourth day, we dry lake 1. There is no full lakes.
After the fifth day, full lakes are [2].
After the sixth day, full lakes are [1,2].
It is easy that this scenario is flood-free. [-1,-1,1,2,-1,-1] is another acceptable scenario.

**Example 3:**
```python
Input: rains = [1,2,0,1,2]
Output: []
```
**Explanation:** After the second day, full lakes are  [1,2]. We have to dry one lake in the third day.
After that, it will rain over lakes [1,2]. It's easy to prove that no matter which lake you choose to dry in the 3rd day, the other one will flood.
 

**Constraints:**

- 1 <= rains.length <= 105
- 0 <= rains[i] <= 109
 



In [ ]:
# Algorithm:
# 1. Keep `full` dict to track lakes currently full.
# 2. Maintain sorted list of `dry_days` (days when rains[i]==0).
# 3. On rain:
#    - If lake already full:
#         * Must find earliest dry day after last rain (binary search)
#         * Assign that dry day to dry this lake
#         * Remove that day from `dry_days`
#    - Else just mark lake full.
# 4. On dry day:
#    - Append to `dry_days` and set ans[i]=1 (temporary)
# 5. Return ans; if flood detected return [].

from bisect import bisect_right, insort

class Solution:
    def avoidFlood(self, rains):
        n = len(rains)
        ans = [-1] * n
        full = {}           # lake -> last rain day
        dry_days = []       # indices where rains[i] == 0

        for i, lake in enumerate(rains):
            if lake > 0:
                if lake in full:
                    idx = bisect_right(dry_days, full[lake])
                    if idx == len(dry_days):
                        return []  # no dry day available -> flood
                    dry_day = dry_days[idx]
                    ans[dry_day] = lake
                    dry_days.pop(idx)
                full[lake] = i
                ans[i] = -1
            else:
                insort(dry_days, i)
                ans[i] = 1  # placeholder if unused

        return ans


In [1]:
# Algorithm:
# 1. Maintain SortedList of indices of dry days.
# 2. Use dictionary to remember last time each lake was filled.
# 3. When a lake is about to flood, find the smallest dry day index > last_rain_day.
# 4. Assign that dry day to dry the lake.
# 5. Continue until all days processed, else return [] if no valid dry day exists.

from sortedcontainers import SortedList

class Solution:
    def avoidFlood(self, rains):
        n = len(rains)
        ans = [-1] * n
        full = {}
        dry_days = SortedList()

        for i, lake in enumerate(rains):
            if lake > 0:
                if lake in full:
                    idx = dry_days.bisect_right(full[lake])
                    if idx == len(dry_days):
                        return []  # flood unavoidable
                    dry_day = dry_days[idx]
                    ans[dry_day] = lake
                    dry_days.remove(dry_day)
                full[lake] = i
                ans[i] = -1
            else:
                dry_days.add(i)
                ans[i] = 1  # placeholder

        return ans


In [ ]:
# Algorithm:
# 1. Preprocess rains -> for each lake, store all days it will rain (queue).
# 2. Iterate days:
#    - If rain on lake:
#         * Pop this day from its queue
#         * If already full -> flood -> return []
#         * Mark lake full
#         * If it will rain again, push (next_rain_day, lake) to heap
#    - If dry day:
#         * Dry the lake whose next rain day is earliest (heap top)
#         * Remove it from full
#         * Mark ans[i] = lake dried
# 3. If no flood -> return ans.

import heapq
from collections import defaultdict, deque

class Solution:
    def avoidFlood(self, rains):
        n = len(rains)
        ans = [-1] * n
        lake_days = defaultdict(deque)
        for i, lake in enumerate(rains):
            if lake > 0:
                lake_days[lake].append(i)

        heap = []  # (next_rain_day, lake)
        full = set()

        for i, lake in enumerate(rains):
            if lake > 0:
                lake_days[lake].popleft()
                if lake in full:
                    return []
                full.add(lake)
                if lake_days[lake]:
                    heapq.heappush(heap, (lake_days[lake][0], lake))
                ans[i] = -1
            else:
                if heap:
                    next_day, lake_to_dry = heapq.heappop(heap)
                    full.remove(lake_to_dry)
                    ans[i] = lake_to_dry
                else:
                    ans[i] = 1  # dry any lake (placeholder)

        return ans
